## Whisper ഉപയോഗിച്ചുള്ള ഇന്ററാക്ടീവ് Phi 3 Mini 4K ഇൻസ്ട്രക്റ്റ് ചാറ്റ്ബോട്ട്

### പരിചയം:
Interactive Phi 3 Mini 4K Instruct Chatbot ഒരു ഉപകരണമാണ്, ഇത് ഉപയോക്താക്കളെ ടെക്സ്റ്റ് അല്ലെങ്കിൽ ഓഡിയോ ഇൻപുട്ട് ഉപയോഗിച്ച് Microsoft Phi 3 Mini 4K instruct ഡെമോവുമായി സംവദിക്കാൻ അനുവദിക്കുന്നു. ചാറ്റ്ബോട്ട് വിവർത്തനം, കാലാവസ്ഥ അപ്‌ഡേറ്റുകൾ, അടിസ്ഥാനപരമായ വിവര ശേഖരണം തുടങ്ങിയ വിവിധ തരം ജോലികൾക്കായി ഉപയോഗിക്കാൻ സാധിക്കുന്നു.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[നിങ്ങളുടെ Huggingface ആക്‌സസ് ടോക്കൺ സൃഷ്ടിക്കുക](https://huggingface.co/settings/tokens)

ഒരു പുതിയ ടോക്കൺ സൃഷ്ടിക്കുക 
ഒരു പുതിയ പേര് നൽകുക 
എഴുത്ത് അനുമതികൾ തിരഞ്ഞെടുക്കുക
ടോക്കൺ കോപ്പി ചെയ്ത് സുരക്ഷിത സ്ഥലത്ത് സംരക്ഷിക്കുക


താഴെയുള്ള Python കോഡ് രണ്ട് പ്രധാന പ്രവർത്തനങ്ങൾ നിർവഹിക്കുന്നു: `os` മോഡ്യൂൾ ഇറക്കുമതി ചെയ്യുക ಮತ್ತು ഒരു എൻവയോൺമെന്റ് വേരിയബിൾ സജ്ജീകരിക്കുക.

1. Importing the `os` module:
   - Python-ൽ ഉള്ള `os` മോഡ്യൂൾ ഓപ്പറേറ്റിംഗ് സിസ്റ്റവുമായി ഇടപഴകാൻ ഒരു മാർഗം നൽകുന്നു. ഇത് എൻവയോൺമെന്റ് വേരിയബിളുകൾ ആക്‌സസ്സ് ചെയ്യുക, ഫയലുകളുമായി, ഡയറക്ടറികളുമായി പ്രവർത്തിക്കുക തുടങ്ങിയ ഓപ്പറേറ്റിംഗ് സിസ്റ്റം സംബന്ധിയായ വിവിധ പ്രവർത്തനങ്ങൾ ചെയ്യാൻ സഹായിക്കുന്നു.
   - ഈ കോഡിൽ, `os` മോഡ്യൂൾ `import` സ്റ്റേറ്റ്മെന്റ് ഉപയോഗിച്ച് ഇറക്കുമതി ചെയ്യുന്നു. ഈ സ്റ്റേറ്റ്മെന്റ് `os` മോഡ്യൂളിന്റെ ഫംഗ്ഷണാലിറ്റി നിലവിലെ Python സ്ക്രിപ്റ്റിൽ ഉപയോഗിക്കാൻ ലഭ്യമാക്കുന്നു.

2. Setting an environment variable:
   - ഒരു എൻവയോൺമെന്റ് വേരിയബിൾ ഓപ്പറേറ്റിംഗ് സിസ്റ്റത്തിൽ പ്രവർത്തിക്കുന്ന പ്രോഗ്രാമുകൾ ആക്‌സസ് ചെയ്യാവുന്ന ഒരു മൂല്യമാണ്. ഇത് കോൺഫിഗറേഷൻ ക്രമീകരണങ്ങൾ അല്ലെങ്കിൽ പല പ്രോഗ്രാമുകളും ഉപയോഗിക്കാവുന്ന മറ്റ് വിവരങ്ങൾ സംഭരിക്കാൻ ഉപയോഗിക്കുന്ന ഒരു മാർഗമാണ്.
   - ഈ കോഡിൽ, പുതിയ ഒരു environment variable `os.environ` ഡിക്ഷണറി ഉപയോഗിച്ച് സജ്ജീകരിക്കുന്നു. ഡിക്ഷണറിയുടെ കീ `'HF_TOKEN'` ആണ്, മൂല്യം `HUGGINGFACE_TOKEN` വേരിയബിൾ നിന്നാണ് നിയോഗിക്കപ്പെടുന്നത്.
   - `HUGGINGFACE_TOKEN` വേരിയബിൾ ഈ കോഡ് സ്നിപ്പറ്റിന് മുകളിൽ നിർവചിച്ചിരിക്കുന്നു, കൂടാതെ അത് `#@param` syntax ഉപയോഗിച്ച് `"hf_**************"` എന്ന ಸ್ಟ್ರിംഗ് മൂല്യമായി നിയോഗിച്ചിരിക്കുന്നു. ഈ സിന്താക്സ് സാധാരണയായി Jupyter notebooks-ൽ ഉപയോക്തൃ ഇൻപുട്ടും പാരാമീറ്റർ കോൺഫിഗറേഷനും നേരിട്ട് നോട്ട്ബുക്ക് ഇന്റർഫേസിൽ അനുവദിക്കാൻ ഉപയോഗിക്കുന്നു.
   - `'HF_TOKEN'` എൻവയോൺമെന്റ് വേരിയബിൾ സജ്ജീകരിക്കുന്നത് വഴി, അത് പ്രോഗ്രാമിന്റെ മറ്റ് ഭാഗങ്ങൾ അല്ലെങ്കിൽ അതേ ഓപ്പറേറ്റിംഗ് സിസ്റ്റത്തിൽ പ്രവർത്തിക്കുന്ന മറ്റ് പ്രോഗ്രാമുകൾ ആക്‌സസ് ചെയ്യാൻ കഴിയും.

Overall, this code imports the `os` module and sets an environment variable named `'HF_TOKEN'` with the value provided in the `HUGGINGFACE_TOKEN` variable.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

ഈ കോഡ് സ്നിപ്പെറ്റ് Jupyter Notebook അല്ലെങ്കിൽ IPython-ൽ ഉള്ള നിലവിലെ സെല്ലിന്റെ ഔട്ട്പുട്ട് ക്ലിയർ ചെയ്യാൻ ഉപയോഗിക്കുന്ന clear_output എന്നൊരു ഫംഗ്ഷൻ നിർവചിക്കുന്നു. നമുക്ക് കോഡ് വിച്ഛേദിച്ച് അതിന്റെ പ്രവർത്തനം മനസിലാക്കാം:

clear_output ഫങ്ഷൻ wait എന്നു പേരുള്ള ഒരു പാരാമീറ്റർ സ്വീകരിക്കുന്നു, ഇത് ഒരു ബൂളിയൻ മൂല്യമാണ്. ഡീഫോൾട്ടായി, wait False ആയി സജ്ജീകരിച്ചിരിക്കുന്നു. ഈ പാരാമീറ്റർ ക്ലിയർ ചെയ്യുന്നതിന് മുമ്പ് പുതിയ ഔട്ട്പുട്ട് ലഭിച്ച് അതോടെ നിലവിലുള്ള ഔട്ട്പുട്ടിന് പകരം അത് വരുമോ എന്നതുവരെ കാത്തിരിക്കണോ എന്നത് നിർണ്ണയിക്കുന്നു.

ഫംഗ്ഷൻ തന്നെ നിലവിലെ സെല്ലിന്റെ ഔട്ട്പുട്ട് ക്ലിയർ ചെയ്യാനാണ് ഉപയോഗിക്കുന്നത്. Jupyter Notebook അല്ലെങ്കിൽ IPython-യിൽ, ഒരു സെൽ പ്രിന്റ് ചെയ്ത ടെക്സ്റ്റ് അല്ലെങ്കിൽ ഗ്രാഫിക്കൽ പ്ലോട്ടുകൾ പോലുള്ള ഔട്ട്പുട്ട് ഉൽപ്പാദിപ്പിക്കുമ്പോൾ ആ ഔട്ട്പുട്ട് സെല്ലിന് താഴെ പ്രദർശിപ്പിച്ചിരിക്കുന്നു. clear_output ഫംഗ്ഷൻ ആ ഔട്ട്പുട്ട് നീക്കം ചെയ്യാൻ അനുവദിക്കുന്നു.

ഫംഗ്ഷന്റെ ഇംപ്ലിമെന്റേഷൻ കോഡ് സ്നിപ്പെറ്റിൽ നൽകിയിട്ടില്ല, എലിപ്സിസ് (...) വഴി സൂചിപ്പിച്ചിരിക്കുന്നതുപോലെ. ആ എലിപ്സിസ് ഔട്ട്പുട്ട് ക്ലിയർ ചെയ്യാനുള്ള യഥാർത്ഥ കോഡിനുള്ള പ്ലേസ്‌ഹോൾഡറായി പ്രവർത്തിക്കുന്നു. ഫംഗ്ഷന്റെ ഇംപ്ലിമെന്റേഷൻ സെലിൽ നിന്നുള്ള നിലവിലെ ഔട്ട്പുട്ട് നീക്കം ചെയ്യുന്നതിനായി Jupyter Notebook അല്ലെങ്കിൽ IPython API-യുമായി ഇടപെടൽ ആവശ്യമാകാം.

മൊത്തത്തിൽ, ഈ ഫങ്ഷൻ Jupyter Notebook അല്ലെങ്കിൽ IPython-ലെ നിലവിലെ സെല്ലിന്റെ ഔട്ട്പുട്ട് ക്ലിയർ ചെയ്യുന്നതിനുള്ള സൗകര്യപ്രദമായ മാർഗം നൽകുന്നു, ഇന്ററാക്ടീവ് കോഡിംഗ് സെഷനുകൾക്കിടെ പ്രദർശിപ്പിച്ച ഔട്ട്പുട്ട് നിയന്ത്രിക്കുകയും അപ്ഡേറ്റ് ചെയ്യുകയും ചെയ്യുന്നത് എളുപ്പമാക്കുന്നു.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Perform text-to-speech (TTS) using the Edge TTS service. Let's go through the relevant function implementations one by one:

1. `calculate_rate_string(input_value)`: ഈ ഫംഗ്‌ഷൻ ഒരു ഇൻപുട്ട് മൂല്യം സ്വീകരിച്ച് TTS വോയ്സിനുള്ള rate string കണക്കാക്കി നൽകുന്നു. ഇൻപുട്ട് മൂല്യം സംസാരത്തിന്റെ ആഗ്രഹിച്ച വേഗത പ്രതിനിധീകരിക്കുന്നു; എവിടെ 1 എന്നത് സാധാരണ വേഗതയിലാണ്. ഫംഗ്ഷൻ rate string കണക്കാക്കിയാൽ, ഇൻപുട്ട് മൂല്യത്തിൽ നിന്ന് 1 കുറയ്ക്കുകയും, അതിനെ 100 ഉപയോഗിച്ച് ഗുണിക്കുകയും ചെയ്യുന്നു, തുടർന്ന് input_value 1 ആകോ അതിന്‍െറത് തുല്യമായോ അതിലധികമായോ ആണോ എന്നു നോക്കി സൈൻ നിർണയിക്കുന്നു. ഫംഗ്ഷൻ "{sign}{rate}" ഫോർമാറ്റിലുള്ള rate string തിരിച്ചു നൽകുന്നു.

2.`make_chunks(input_text, language)`: ഈ ഫംഗ്ഷൻ ഒരു ഇൻപുട്ട് ടെക്സ്റ്റും ഒരു ഭാഷയും പാരാമീറ്റർ ആയി സ്വീകരിക്കുന്നു. ഇത് ഭാഷാനുസൃത നിയമങ്ങൾക്കനുസരിച്ച് ഇൻപുട്ട് ടെക്സ്റ്റിനെ ചെറിയ ഭാഗങ്ങളായി (chunks) വിഭജിക്കുന്നു. ഈ നടപ്പിൽ, language "English" ആയാൽ, ഫംഗ്ഷൻ ടെക്സ്റ്റിനെ ഓരോ മുഴുവൻവിരാമത്തിലും (".") വിഭജിക്കുകയും ഇവിടെയുള്ള മുൻവശവും പിന്നിലൂടെ ഉള്ള വയിടുകൾ (whitespace) നീക്കം ചെയ്യുകയും ചെയ്യുന്നു. പിന്നീട് ഓരോ chunk നും ഒരു പൂരിപ്പിക്കലായി ഒരു പിരിയോദം ചേര്ക്കുകയും, ഫിൽട്ടർ ചെയ്‌ത chunk-കളുടെ ലിസ്റ്റ് തിരിച്ചു നൽകുകയും ചെയ്യുന്നു.

3. `tts_file_name(text)`: ഈ ഫംഗ്ഷൻ ഇൻപുട്ട് ടെക്സ്റ്റിനെ അടിസ്ഥാനമാക്കി TTS ഓഡിയോ ഫയലിനുള്ള ഒരു പേര് സൃഷ്ടിക്കുന്നു. ഇത് ടെക്സ്റ്റിൽ നിരവധി പരിവർത്തനങ്ങൾ ചെയ്യുന്നു: അവസാനം ഒരു പിരിയോദം ഉണ്ടെങ്കിൽ അത് നീക്കം ചെയ്യൽ, ടെക്സ്റ്റ് lowercase ആയി മാറ്റൽ, മുൻവശവും പിന്നെയുള്ള whitespace കളെ ചുരുക്കൽ, തരംതിരിക്കാനുള്ള സ്ഥലങ്ങൾ underscores-ആയി മാറ്റൽ എന്നിവ. ശേഷം ടെക്സ്റ്റ് 25 അക്ഷരത്തിൽപര്യമാകുന്നവിധം (ന്യൂനമല്ലെങ്കിൽ)ചുരുക്കുകയും അല്ലെങ്കിൽ ടെക്സ്റ്റ് ശൂന്യമായിരുന്നാൽ മുഴുവൻ വാക്യം ഉപയോഗിക്കുകയും ചെയ്യുന്നു. ഒടുവിൽ ഇത് [`uuid`] മോഡ്യൂൾ ഉപയോഗിച്ച് ഒരു യാദൃച്ചിക സ്ട്രിംഗ് സൃഷ്‌ടിക്കുകയും അത് ചുരുക്കിയ ടെക്സ്റ്റിനോടൊപ്പം ചേർത്ത് "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3" എന്ന ഫോർമാറ്റിലുള്ള ഫയൽനാമം ഉണ്ടാക്കുകയും ചെയ്യുന്നു.

4. `merge_audio_files(audio_paths, output_path)`: ഈ ഫംഗ്ഷൻ ഒന്നിലധികം ഓഡിയോ ഫയലുകൾ ഒരൊറ്റ ഓഡിയോ ഫയലായി ലയിപ്പിക്കുന്നു. ഇത് ഓഡിയോ ഫയൽ പാത്തുകളുടെ ഒരു ലിസ്റ്റും ഒരു output_path ഉം പാരാമീറ്ററുകളായി സ്വീകരിക്കുന്നു. ഫംഗ്ഷൻ ആദ്യം ഒരു ശൂന്യ `AudioSegment` ഒബ്‌ജക്റ്റ് എന്ന [`merged_audio`] ആരംഭിക്കുന്നു. തുടർന്ന് ഓരോ ഓഡിയോ ഫയൽ പാത്തിലും സഞ്ചരിച്ച്, `pydub` ലൈബ്രറിയിലെ `AudioSegment.from_file()` മെതഡ് ഉപയോഗിച്ച് ആ ഫയൽ ലോഡ് ചെയ്യുകയും നിലവിലെ ഓഡിയോ ഫയൽ [`merged_audio`] ഒബ്‌ജക്ടിലേക്ക് ചേർക്കുകയും ചെയ്യുന്നു. അവസാനം ലയിച്ച ഓഡിയോ നിർദ്ദിഷ്ട output_path-ിലേക്ക് MP3 ഫോർമാറ്റിൽ എക്സ്പോർട്ട് ചെയ്യുന്നു.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path): This function performs the TTS operation using the Edge TTS service. It takes a list of text chunks, the speed of the speech, the voice name, and the save path as parameters. If the number of chunks is greater than 1, the function creates a directory for storing the individual chunk audio files. It then iterates through each chunk, constructs an Edge TTS command using the `calculate_rate_string()' function, the voice name, and the chunk text, and executes the command using the `os.system()` function. If the command execution is successful, it appends the path of the generated audio file to a list. After processing all the chunks, it merges the individual audio files using the `merge_audio_files()` function and saves the merged audio to the specified save path. If there is only one chunk, it directly generates the Edge TTS command and saves the audio to the save path. Finally, it returns the save path of the generated audio file.

6. `random_audio_name_generate()`: ഈ ഫംഗ്ഷൻ [`uuid`] മോഡ്യൂൾ ഉപയോഗിച്ചുകൊണ്ട് ഒരു യാദൃച്ഛിക ഓഡിയോ ഫയൽ നാമം സൃഷ്ടിക്കുന്നു. ഇത് ഒരു യാദൃച്ഛിക UUID ഉണ്ടാക്കി, അതിനെ String ആക്കി, ആദ്യത്തെ 8 അക്ഷരങ്ങൾ എടുത്ത്, ".mp3" എക്സ്റ്റൻഷൻ ചേർക്കുകയും അതെ തിരിച്ചുനൽകുകയും ചെയ്യുന്നു.

7. `talk(input_text)`: ഈ ഫംഗ്ഷൻ TTS പ്രവർത്തനം നിർവഹിക്കുന്നതിനുള്ള പ്രധാന എൻട്രി പോയിന്റാണ്. ഇത് ഒരു ഇൻപുട്ട് ടെക്സ്റ്റ് പാരാമീറ്ററായി സ്വീകരിക്കുന്നു. ആദ്യം തെക്സ്റ്റിന്റെ നീളം പരിശോധിച്ച് അത് വലിയ വാചകമാണോ (600 അക്ഷരങ്ങൾക്കോ അതിനേക്കാളോ) എന്ന് നിർണ്ണയിക്കുന്നു. നീളം കൂടിയതോ അല്ലയോ എന്നും `translate_text_flag` വ്യത്യാസമുള്ള മൂല്യവും അടിസ്ഥാനമാക്കി, ഭാഷ നിർണയിച്ച് `make_chunks()` ഫംഗ്ഷൻ ഉപയോഗിച്ച് ടെക്സ്റ്റിന്റെ chunk-കളുടെ ലിസ്റ്റ് സൃഷ്ടിക്കുന്നു. പിന്നീട് `random_audio_name_generate()` ഫംഗ്ഷൻ ഉപയോഗിച്ച് ഓഡിയോ ഫയലിനുള്ള ഒരു save path ഉണ്ട് ഉണ്ടാക്കുകയും ഏതു ലോക്കേഷനിൽ സേവ് ചെയ്യാനെന്നോ അത് നിർണയിക്കുകയും ചെയ്യുന്നു. ഒടുവിൽ `edge_free_tts()` ഫംഗ്ഷൻ വിളിച്ച് TTS പ്രവർത്തനം നിർവഹിക്കുകയും സൃഷ്ടിച്ച ഓഡിയോ ഫയലിന്റെ save path തിരികെ നൽകുകയും ചെയ്യുന്നു.

Overall, these functions work together to split the input text into chunks, generate a file name for the audio file, perform the TTS operation using the Edge TTS service, and merge the individual audio files into a single audio file.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

The implementation of two functions: convert_to_text and run_text_prompt, as well as the declaration of two classes: str and Audio.

The convert_to_text function takes an audio_path as input and transcribes the audio to text using a model called whisper_model. The function first checks if the gpu flag is set to True. If it is, the whisper_model is used with certain parameters such as word_timestamps=True, fp16=True, language='English', and task='translate'. If the gpu flag is False, the whisper_model is used with fp16=False. The resulting transcription is then saved to a file named 'scan.txt' and returned as the text.

The run_text_prompt function takes a message and a chat_history as input. It uses the phi_demo function to generate a response from a chatbot based on the input message. The generated response is then passed to the talk function, which converts the response into an audio file and returns the file path. The Audio class is used to display and play the audio file. The audio is displayed using the display function from the IPython.display module, and the Audio object is created with the autoplay=True parameter, so the audio starts playing automatically. The chat_history is updated with the input message and the generated response, and an empty string and the updated chat_history are returned.

The str class is a built-in class in Python that represents a sequence of characters. It provides various methods for manipulating and working with strings, such as capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, and more. These methods allow you to perform operations like searching, replacing, formatting, and manipulating strings.

The Audio class is a custom class that represents an audio object. It is used to create an audio player in the Jupyter Notebook environment. The class accepts various parameters such as data, filename, url, embed, rate, autoplay, and normalize. The data parameter can be a numpy array, a list of samples, a string representing a filename or URL, or raw PCM data. The filename parameter is used to specify a local file to load the audio data from, and the url parameter is used to specify a URL to download the audio data from. The embed parameter determines whether the audio data should be embedded using a data URI or referenced from the original source. The rate parameter specifies the sampling rate of the audio data. The autoplay parameter determines whether the audio should start playing automatically. The normalize parameter specifies whether the audio data should be normalized (rescaled) to the maximum possible range. The Audio class also provides methods like reload to reload the audio data from file or URL, and attributes like src_attr, autoplay_attr, and element_id_attr to retrieve the corresponding attributes for the audio element in HTML.

Overall, these functions and classes are used to transcribe audio to text, generate audio responses from a chatbot, and display and play audio in the Jupyter Notebook environment.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ഡിസ്‌ക്ലെയിമര്‍: 

ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. ഞങ്ങൾ കൃത്യതയ്ക്ക് പരിശ്രമിച്ചെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകളും അപൂർണ്ണതകളും ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. ആദിമ ഭാഷയിലെ (മൂല) പ്രമാണം ആധികാരിക ഉറവിടമായി കണക്കാക്കപ്പെടണം. നിർണ്ണായക വിവരങ്ങൾക്ക് പ്രൊഫഷണൽ മനുഷ്യവിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിച്ചതിൽ നിന്നുണ്ടാവുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കും തെറ്റായി വ്യാഖ്യാനിക്കുന്നതുമടക്കമുള്ള കാര്യങ്ങൾക്ക് നമ്മൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
